In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import sklearn
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.ensemble import RandomForestClassifier
from sklearn import tree

In [2]:
final = pd.read_csv('temp.csv')

In [4]:
final = final.drop(columns=['Unnamed: 0'])

In [5]:
final.columns

Index(['Year', 'Player Name', 'Pos', 'Age', 'Team', 'Lg', 'Level', 'Org', 'G',
       'AB', 'R', 'H', '2B', '3B', 'HR', 'RBI', 'SB', 'CS', 'BB', 'SO', 'HBP',
       'SH', 'SF', 'IBB', 'GDP', 'AVG', 'OBP', 'SLG', 'OPS', 'TB', 'XBH', 'PA',
       'HiLvl', 'Draft Information', 'ISO', 'BB%', 'K%', 'SB%'],
      dtype='object')

In [7]:
final['BABIP'] = (final.H - final.HR) / (final.AB - final.SO - final.HR + final.SF)

In [36]:
final['SB%'].fillna(0, inplace=True)

In [37]:
look = final.groupby(['Lg', 'Level', 'Year']).transform(lambda x: (x - x.mean()) / x.std())


In [38]:
look

,Age,G,AB,R,H,2B,3B,HR,RBI,SB,...,SLG,OPS,TB,XBH,PA,ISO,BB%,K%,SB%,BABIP
0,0.790219,1.557910,1.892788,1.421280,2.284221,2.365349,0.578911,1.824354,2.071115,-0.267658,...,0.995738,0.879279,2.283855,2.219817,1.755493,0.706556,-0.813167,-1.414854,0.343148,0.264515
1,-0.721435,1.321281,1.510033,1.465381,1.819218,1.899671,2.490075,0.247031,1.103400,2.085246,...,0.585996,0.635377,1.602209,1.549081,1.476674,0.177539,-0.206077,-1.019719,0.558810,0.603249
2,-0.116773,0.663979,0.925461,1.421280,1.011583,1.433993,-0.376671,1.193425,1.103400,1.039511,...,0.760136,0.635377,1.132631,1.280786,0.906645,0.736785,-0.229868,-0.791043,0.343148,-0.062822
3,-0.116773,0.926900,0.960257,0.627454,0.571055,0.735476,2.012284,-0.383899,0.311633,1.431662,...,-0.469089,-0.612831,0.466132,0.542977,0.906645,-0.321249,-0.544863,0.067166,0.805281,-0.344578
4,-0.116773,1.032069,0.967216,0.098237,0.864741,0.386217,0.101120,0.404763,0.751504,-0.659809,...,-0.059348,-0.268498,0.708495,0.408830,0.850881,-0.170101,-1.276435,-0.862132,-0.196007,-0.385701
5,-1.023766,0.663979,0.848910,0.759758,0.693424,0.502637,-0.854462,2.770748,1.411309,-0.529092,...,1.087930,0.592335,1.238665,1.347860,0.745549,1.628557,-0.757740,0.924125,-0.196007,-0.403582
6,-0.419104,0.637687,0.661012,0.583352,0.766845,0.386217,0.101120,-0.068434,-0.084250,0.124492,...,0.053331,0.176266,0.511575,0.207609,0.646414,-0.336364,-0.268813,-0.635655,0.245120,0.445802
7,-0.721435,0.217014,0.445277,0.495150,0.399739,0.153378,1.056702,1.351157,0.575555,0.385926,...,0.913789,0.699939,0.723643,0.811271,0.429555,1.235573,0.024648,1.609079,0.127486,0.768276
8,-1.023766,0.611395,0.438318,-0.078169,0.252896,0.269798,0.101120,-0.383899,0.047711,-0.398375,...,-0.448602,-0.670220,0.087440,0.006388,0.324223,-0.502626,-1.142313,-0.561457,-0.196007,-0.478675
9,0.185558,-0.072199,0.146032,0.274642,0.155001,-0.195880,1.534493,-0.383899,-0.084250,0.255209,...,0.022601,0.248002,0.057144,-0.060686,0.212696,-0.154986,0.639518,-0.882426,0.539205,-0.007985


In [21]:
look.columns

Index(['Age', 'G', 'AB', 'R', 'H', '2B', '3B', 'HR', 'RBI', 'SB', 'CS', 'BB',
       'SO', 'HBP', 'SH', 'SF', 'IBB', 'GDP', 'AVG', 'OBP', 'SLG', 'OPS', 'TB',
       'XBH', 'PA', 'ISO', 'BB%', 'K%', 'SB%', 'BABIP'],
      dtype='object')

In [39]:
total = final[['Year', 'Player Name', 'Pos', 'Team', 'Lg', 'Level', 'Org', 'PA',
       'HiLvl', 'Draft Information']]

In [40]:
total[['Age', 'G', 'AB', 'R', 'H', '2B', '3B', 'HR', 'RBI', 'SB', 'CS', 'BB',
       'SO', 'HBP', 'SH', 'SF', 'IBB', 'GDP', 'AVG', 'OBP', 'SLG', 'OPS', 'TB',
       'XBH', 'ISO', 'BB%', 'K%', 'SB%', 'BABIP']] = look[['Age', 'G', 'AB', 'R', 'H', '2B', '3B', 'HR', 'RBI', 'SB', 'CS', 'BB',
       'SO', 'HBP', 'SH', 'SF', 'IBB', 'GDP', 'AVG', 'OBP', 'SLG', 'OPS', 'TB',
       'XBH', 'ISO', 'BB%', 'K%', 'SB%', 'BABIP']]

In [42]:
total.to_csv('perm.csv')